In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bokeh.plotting import figure
from bokeh.io import show, output_notebook, output_file, push_notebook

from bokeh.models import ColumnDataSource, HoverTool, Legend, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RadioGroup, Tabs

from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

In [2]:
NPS = pd.read_csv('bokeh_app/data/NPS.tsv', na_values=['-9', '-2', '-1', '-8'], sep='\t')
NPS

,YEAR,STATEID,STATE,REGION,CUSGT1M,CUSGT1F,CUSLT1M,CUSLT1F,CUSUNSM,CUSUNSF,...,HIVASYMT,HIVLESSM,HIVLESSF,HIVLESST,AIDSCONM,AIDSCONF,AIDSCONT,HIVATOTM,HIVATOTF,HIVATOTT
0,1978,1,AL,3,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
1,1978,2,AK,4,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
2,1978,4,AZ,4,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
3,1978,5,AR,3,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
4,1978,6,CA,4,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
5,1978,8,CO,4,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
6,1978,9,CT,1,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
7,1978,10,DE,3,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
8,1978,11,DC,3,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
9,1978,12,FL,3,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,


In [38]:
Totals = NPS.loc[NPS['STATEID'] == 99, ['YEAR', 'CUSGT1M']]
totals = NPS.query('STATEID == 99 and YEAR >= 1983')

""""
totals[['YEAR','RLUNEXPM', 'RLUNCOMM', 'RLUNOTHM',
        'RLCOPROM', 'RLCOSUPM', 'RLCODPM', 'RLCOOTHM']]
"""
def make_parole(state):
    totals = NPS.query('STATEID == ' + state + 'and YEAR >= 1983')
    return ColumnDataSource(totals)


In [41]:
# RELEASE GRAPH ON RELEASES

src = make_parole('0')


m = figure(plot_width = 800, plot_height = 600,
            title = "National Prisoner Release Statistics",
              x_axis_label = 'Year', y_axis_label = 'Number of Prisoners')

# Male
# RLUNEXPM: UNCONDITIONAL RELEASE EXPIRATIONS OF SENTENCE, MALE
rm = m.circle(src.data['YEAR'], src.data['RLUNEXPM'],
         size = 7, color = 'navy', alpha = 0.5,
         hover_fill_alpha = 1.0, hover_fill_color = 'navy')

# RLCOSUPM: SUPERVISED MANDATORY RELEASE, MALE
sm = m.circle(src.data['YEAR'], src.data['RLCOSUPM'],
         size = 7, color = 'red', alpha = 0.5,
         hover_fill_alpha = 1.0, hover_fill_color = 'red')

# RLCODPM: DISCRETIONARY PAROLE, MALE
pm = m.circle(src.data['YEAR'], src.data['RLCODPM'],
         size = 7, color = 'green', alpha = 0.5,
         hover_fill_alpha = 1.0, hover_fill_color = 'green')

# Female
# RLUNEXPM: UNCONDITIONAL RELEASE EXPIRATIONS OF SENTENCE, FEMALE
rf = m.square(src.data['YEAR'], src.data['RLUNEXPF'],
         size = 7, color = 'navy', alpha = 0.5,
         hover_fill_alpha = 1.0, hover_fill_color = 'navy')

# RLCOSUPM: SUPERVISED MANDATORY RELEASE, FEMALE
sf = m.square(src.data['YEAR'], src.data['RLCOSUPF'],
         size = 7, color = 'red', alpha = 0.5,
         hover_fill_alpha = 1.0, hover_fill_color = 'red')

# RLCODPM: DISCRETIONARY PAROLE, FEMALE
pf = m.square(src.data['YEAR'], src.data['RLCODPF'],
         size = 7, color = 'green', alpha = 0.5,
         hover_fill_alpha = 1.0, hover_fill_color = 'green')

# adding tooltips
h = HoverTool(tooltips = [
    ('Year', '@x'),
    ('# of Prisoners', '@y')
    ])

m.add_tools(h)

# Legend
legendm = Legend(items=[
    ('Male, Unconditional Release', [rm]),
    ('Male, Supervised Release', [sm]),
    ('Male, Parole', [pm]),
    ('Female, Unconditional Release', [rf]),
    ('Female, Supervised Release', [sf]),
    ('Female, Parole', [pf])
])

m.add_layout(legendm, 'right')
m.legend.click_policy='hide'
show(m)


In [5]:
sentenceLength = NPS.loc[NPS['STATEID'] == 99, ['YEAR', 'JURGT1M', 'JURLT1M', 'JURGT1F', 'JURLT1F']]
sentenceL = ColumnDataSource(sentenceLength)

In [6]:
# National Prisoner Sentence Lengths
l = figure(plot_width = 800, plot_height = 600,
            title = "Received Sentence Lengths",
              x_axis_label = 'Year', y_axis_label = 'Number of Prisoners')


l1 = l.circle(sentenceL.data['YEAR'], sentenceL.data['JURGT1M'],
         size = 7, color = 'navy', alpha = 0.5,
         hover_fill_alpha = 1.0, hover_fill_color = 'navy')

l2 = l.circle(sentenceL.data['YEAR'], sentenceL.data['JURLT1M'],
         size = 7, color = 'red', alpha = 0.5,
         hover_fill_alpha = 1.0, hover_fill_color = 'red')

l3 = l.circle(sentenceL.data['YEAR'], sentenceL.data['JURGT1F'],
         size = 7, color = 'green', alpha = 0.5,
         hover_fill_alpha = 1.0, hover_fill_color = 'green')

l4 = l.circle(sentenceL.data['YEAR'], sentenceL.data['JURLT1F'],
         size = 7, color = 'purple', alpha = 0.5,
         hover_fill_alpha = 1.0, hover_fill_color = 'purple')


# adding tooltips
h = HoverTool(tooltips = [
    ('Year', '@x'),
    ('# of Prisoners', '@y')
    ])

l.add_tools(h)

# Legend
legendl = Legend(items=[
    ('Male, More than 1 Year', [l1]),
    ('Male, Less than Year', [l2]),
    ('Female, More than 1 Year', [l3]),
    ('Female, Less than 1 Year', [l4])
])

l.add_layout(legendl, 'right')
l.legend.click_policy='hide'

show(l)

In [45]:
# Make Dataset for all States per Year
def make_dataset(variables, year = 2000):
    new_src = NPS.query('STATEID < 60 and STATEID != 52 and STATEID != 11')
    #variables.append('YEAR')
    variables.append('STATE')
    new_src = new_src.loc[new_src['YEAR'] == year, variables]
    #new_src['STATEID'] = list(range(1, 51))
    
    #new_src['RIGHT'] = new_src['STATEID'] - 1
    #new_src['LEFT'] = new_src['STATEID']
    
    #state_list = new_src['STATE']
    #new_src = new_src.T
    #new_src.columns = state_list
    new_src = new_src.set_index('STATE')
    return ColumnDataSource(new_src)

make_dataset(['ASIANM', 'ASIANF', 'WHITEM', 'WHITEF'], 2000).data

{'STATE': array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
        'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
        'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
        'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
        'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object),
 'ASIANM': array([3.000e+00, 8.200e+01, 3.550e+02, 7.000e+00, 1.209e+03, 1.250e+02,
        4.900e+01, 2.000e+00, 5.000e+01, 4.900e+01, 9.320e+02, 2.600e+01,
        8.500e+01, 2.700e+01, 6.100e+01, 6.800e+01, 7.000e+00, 3.000e+00,
        4.000e+00, 2.100e+01, 1.170e+02, 8.300e+01, 1.400e+01, 2.100e+01,
        4.900e+01, 5.000e+00, 1.900e+01, 1.280e+02, 1.400e+01, 8.800e+01,
        7.000e+00, 3.420e+02, 6.700e+01, 6.000e+00, 4.300e+01, 3.300e+01,
        1.320e+02, 1.510e+02, 3.600e+01, 1.300e+01, 0.000e+00, 3.300e+01,
        8.000e+00, 8.700e+01, 9.000e+00, 1.010e+02, 3.710e+02, 1.000e+00,
        1.280e+02, 4.000e+00]),
 

In [22]:
# Create the plot
demographic = make_dataset(['ASIANM', 'ASIANF', 'WHITEF'], 2000)

bp = figure(plot_width = 1200, plot_height = 600,
            title = "Demographics Per Year",
            x_range = demographic.data['STATE'],
            x_axis_label = 'State', y_axis_label = 'Number of Prisoners')

states = demographic.data['STATE']
races = list()
colors = list()
for i, x in enumerate(demographic.data):
    if x == 'index' or x == 'STATE':
        continue
    races.append(x)
    colors.append(Category20_16[i])
    
races_legend = list()
for name in races:
    if name == 'ASIANM':
        races_legend.append('Asian, Male') 
    elif name == 'ASIANF':
        races_legend.append('Asian, Female')
    elif name == 'WHITEF':
        races_legend.append('White, Female')
    elif name == 'WHITEM':
        races_legend.append('White, Male')
    elif name == 'BLACKF':
        races_legend.append('Black, Female')
    elif name == 'BLACKM':
        races_legend.append('Black, Male')
    elif name == 'HISPM':
        races_legend.append('Hispanic/Latino, Male')
    elif name == 'HISPF':
        races_legend.append('Hispanic/Latino, Female')

AM = bp.vbar_stack(races, x = 'STATE', width = 0.9, color = colors, source = demographic.data,
                   legend = races_legend)
    
#AM = bp.vbar(x = demographic.data['STATE'], top = demographic.data['ASIANM'], width = 0.9, color=Category20_16[15])

#AF = bp.vbar(x = demographic.data['STATE'], top = demographic.data['ASIANF'], width = 0.9, color=Category20_16[14])
#AM = bp.quad(source = demographic, bottom = 0, top = 'ASIANM',
#             left = 'LEFT', right = 'RIGHT',
#            line_color = 'black')

#AM = bp.quad(source = demographic, bottom = 0, top = 'ASIANF',
#             left = 'LEFT', right = 'RIGHT',
#            line_color = 'black', color = Category20_16[15])

# adding tooltips
for race in races:
    h = HoverTool(tooltips = [
    ('State', '@STATE'),
    ('# of Prisoners', '@races')
    ], names = races)

    bp.add_tools(h)

bp.legend.location = 'top_right'
# Legend
#legendl = Legend(items=[
#    ('Asian, Male', [AM])
#])

#bp.add_layout(legendl, 'right')
bp.legend.click_policy='hide'

show(bp)

In [44]:
def make_plot(src):
    bp = figure(plot_width = 1200, plot_height = 600,
            title = "Demographics Per Year",
            x_range = src.data['STATE'],
            x_axis_label = 'State', y_axis_label = 'Number of Prisoners')

    states = src.data['STATE']
    races = list()
    colors = list()
    for i, x in enumerate(src.data):
        if x == 'index' or x == 'STATE':
            continue
        races.append(x)
        colors.append(Category20_16[i])

    races_legend = list()
    for name in races:
        if name == 'ASIANM':
            races_legend.append('Asian, Male') 
        elif name == 'ASIANF':
            races_legend.append('Asian, Female')
        elif name == 'WHITEF':
            races_legend.append('White, Female')
        elif name == 'WHITEM':
            races_legend.append('White, Male')
        elif name == 'BLACKF':
            races_legend.append('Black, Female')
        elif name == 'BLACKM':
            races_legend.append('Black, Male')
        elif name == 'HISPM':
            races_legend.append('Hispanic/Latino, Male')
        elif name == 'HISF':
            races_legend.append('Hispanic/Latino, Female')

    AM = bp.vbar_stack(races, x = 'STATE', width = 0.9, color = colors, source = src.data,
                       legend = races_legend)
    for race in races:
        h = HoverTool(tooltips = [
        ('State', '@STATE'),
        ('# of Prisoners', '@races')
        ], names = races)

        bp.add_tools(h)

    bp.legend.location = 'top_right'
    bp.legend.click_policy='hide'
    
    return bp

show(make_plot(make_dataset(['WHITEM', 'WHITEF'], 2000)))

In [10]:
# Interactivity

# Update function
def update(attr, old, new):
    # get list of variables to update
    demo_to_plot = [demo_selection.labels[i] for i in 
                        demo_selection.active]
    
    # update the slider for year
    year = year_select.value
    
    # update the dataset
    new_src = ColumnDataSource(make_dataset(year))
    
    src.data.update(new_src.data)


#create a checkbox
race = ['ASIANM', 'ASIANF']
demo_selection =  CheckboxGroup(labels = race, active=[0, 1])
demo_selection.on_change('active', update)

# create a slider for year
year_select = Slider(start = 1978, end = 2016, step = 1, value = 2000, title = 'Year')
year_select.on_change('value', update)



In [11]:
controls = WidgetBox(demo_selection, year_select)
show(controls)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [19]:
def modify_doc(doc):
    
    # make the dataset
    def make_dataset(variables, year = 2000):
        new_src = src.query('STATEID < 60 and STATEID != 52 and STATEID != 11')
        #variables.append('YEAR')
        variables.append('STATE')
        new_src = new_src.loc[new_src['YEAR'] == year, variables]
        new_src = new_src.set_index('STATE')
        return ColumnDataSource(new_src)

    # make the plot
    def make_plot(src):
        bp = figure(plot_width = 1200, plot_height = 600,
                title = "Demographics Per State",
                x_range = src.data['STATE'],
                x_axis_label = 'State', y_axis_label = 'Number of Prisoners')

        states = src.data['STATE']
        races = list()
        colors = list()
        for i, x in enumerate(src.data):
            if x == 'index' or x == 'STATE':
                continue
            races.append(x)
            colors.append(Category20_16[i])

        races_legend = list()
        for name in races:
            if name == 'ASIANM':
                races_legend.append('Asian, Male') 
            elif name == 'ASIANF':
                races_legend.append('Asian, Female')
            elif name == 'WHITEF':
                races_legend.append('White, Female')
            elif name == 'WHITEM':
                races_legend.append('White, Male')
            elif name == 'BLACKF':
                races_legend.append('Black, Female')
            elif name == 'BLACKM':
                races_legend.append('Black, Male')
            elif name == 'HISPM':
                races_legend.append('Hispanic/Latino, Male')
            elif name == 'HISF':
                races_legend.append('Hispanic/Latino, Female')

        AM = bp.vbar_stack(races, x = 'STATE', width = 0.9, color = colors, source = src.data,
                           legend = races_legend)
        for race in races:
            h = HoverTool(tooltips = [
            ('State', '@STATE'),
            ('# of Prisoners', '@races')
            ], names = races)

            bp.add_tools(h)

        bp.legend.location = 'top_right'
        bp.legend.click_policy='hide'
    
        return bp

    # Update function
    def update(attr, old, new):
        # get list of variables to update
        demo_to_plot = [demo_selection.labels[i] for i in 
                            demo_selection.active]

        # update the slider for year
        year = year_select.value

        # update the dataset
        new_src = make_dataset(demo_to_plot, year)

        src.data.update(new_src.data)


    #create a checkbox
    race = ['ASIANM', 'ASIANF', 'WHITEF', 'WHITEM', 'BLACKM', 'BLACKF', 'HISPM', 'HISPF']
    demo_selection =  CheckboxGroup(labels = race, active=[0, 1])
    demo_selection.on_change('active', update)

    # create a slider for year
    year_select = Slider(start = 1978, end = 2016, step = 1, value = 2000, title = 'Year')
    year_select.on_change('value', update)
    
    # Initialize the plot
    src = make_dataset(['ASIANM', 'ASIANF'], 2000)
    p = make_plot(src)
    
    # add the controls
    controls = WidgetBox(year_select, demo_selection)
    
    # create a layout
    layout = row(controls, p)
    
    # add a tab
    tab = Panel(child = layout, title = 'Prison Admissions by Ethnicity')

handler = FunctionHandler(modify_doc)
app = Application(handler)

In [18]:
show(app)